# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

- ```population``` — the global population.
- ```population_growth``` — the annual population growth rate, as a percentage.
- ```area``` — the total land and water area.

We will use SQL in Jupyter to analyze the data. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Overview of the Data

Let's start with a simple query to return basic information about the database.

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [2]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here is the data dictionary for our ```facts``` database

- ```name``` — the name of the country.
- ```area```— the country's total area (both land and water).
- ```area_land``` — the country's land area in square kilometers.
- ```area_water``` — the country's waterarea in square kilometers.
- ```population``` — the country's population.
- ```population_growth```— the country's population growth as a percentage.
- ```birth_rate``` — the country's birth rate, or the number of births per year per 1,000 people.
- ```death_rate``` — the country's death rate, or the number of death per year per 1,000 people.

## Summary Statistics

In [4]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


A minimum population of 0 must be an error in the data, so let's look into that. Similarly, a population growth of 0 also is suspicious. 

The maximum population of 7.2 billion is about the population of the world.

Let's look into these outliers.

In [10]:
%%sql
SELECT name, population, population_growth
    FROM facts
    WHERE population == 0;

Done.


name,population,population_growth
Antarctica,0,None


In [8]:
%%sql
SELECT name, population, population_growth
    FROM facts
    WHERE population == (SELECT MAX(population)
                         FROM facts);

Done.


name,population,population_growth
World,7256490011,1.08


In [11]:
%%sql
SELECT name, population, population_growth
    FROM facts
    WHERE population_growth == 0;

Done.


name,population,population_growth
Holy See (Vatican City),842,0.0
Cocos (Keeling) Islands,596,0.0
Greenland,57733,0.0
Pitcairn Islands,48,0.0


We see there is are entries for Antarctica and the World, which account for our smallest and largest population. Let's take care to not include these in future analysis. 

There are a few countries with no population growth, mostly ones with very small populations. The exception is Greenland. For all of these, the reported population growth could be a result of rounding.

Let's recalculate the summary statistics we did earlier excluding the World and Antarctica.

In [14]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
    WHERE name <> 'Antarctica'
    AND name <> 'World'; 

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


## Exploring Average Population and Area

In [15]:
%%sql
SELECT AVG(population) AS 'Avg Pop', AVG(area) AS 'Avg Area'
    FROM facts
    WHERE name <> 'Antarctica'
    AND name <> 'World';

Done.


Avg Pop,Avg Area
32377011.0125,555093.546184739


## Finding Densely Populated Countries

In [17]:
%%sql
SELECT name, population, area
    FROM facts
    WHERE population > (SELECT AVG(population)
                       FROM facts
                       WHERE name <> 'Antarctica'
                       AND name <> 'World')
    AND area < (SELECT AVG(area)
               FROM facts
               WHERE name <> 'Antarctica'
               AND name <> 'World');

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


## Which countries will add the most people to their populations next year?

In [26]:
%%sql
SELECT name, population AS 'Current Population',
    ROUND((population_growth * population),0) AS 'Expected New People'
    FROM facts
    WHERE name <> 'World'
    ORDER BY (population_growth * population) DESC
    LIMIT 5;

Done.


name,Current Population,Expected New People
India,1251695584,1527068612.0
China,1367485388,615368425.0
Nigeria,181562056,444827037.0
Pakistan,199085847,290665337.0
Ethiopia,99465819,287456217.0


## Which countries have more water than land?

In [32]:
%%sql
SELECT name, ROUND((CAST(area_water as float) / CAST(area as float)), 3) AS 'Fraction Water'
    FROM facts
    WHERE area_water > area_land;

Done.


name,Fraction Water
British Indian Ocean Territory,0.999
Virgin Islands,0.819
